In [77]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import data_cleaning as dc
from sklearn import metrics
%matplotlib inline

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import train_test_split

In [131]:
df1 = pd.read_csv('estimated_rent.csv')
df2 = pd.read_csv('distance_nearest_subway.csv')
df3 = pd.read_csv('real_estate_price_transport.csv')
df = pd.concat([df1,df2,df3], axis=1, sort=True)

In [105]:
df1_unique_addresses = df1.drop_duplicates('addr')
df2_unique_addresses = df2.drop_duplicates('address')
df3_unique_addresses = df3.drop_duplicates('address')

In [125]:
df

,Unnamed: 0,Unnamed: 0.1,RegionID,neighborhood,addr,sale_date,sale_price,zip_code,rent_estimate,Unnamed: 0,...,commercial_units,gross_square_feet,land_square_feet,neighborhood,residential_units,sale_date,sale_price,total_units,year_built,zip_code
0,0.0,0.0,270958.0,upper west side,360 MANHATTAN AVENUE,2012-05,695000.0,10026.0,2664.0,0.0,...,0,1428,2513,BATH BEACH,1,2012-06-28,610000,1,1930,11228
1,1.0,1.0,270958.0,upper west side,360 MANHATTAN AVENUE,2012-03,665000.0,10026.0,2575.0,1.0,...,0,1724,4833,BATH BEACH,1,2012-12-18,860000,1,1930,11214
2,2.0,2.0,270958.0,upper west side,360 MANHATTAN AVENUE,2012-02,721000.0,10026.0,2796.0,2.0,...,0,0,0,BATH BEACH,1,2012-07-25,452000,1,0,11214
3,3.0,3.0,270958.0,upper west side,360 MANHATTAN AVENUE,2013-08,955000.0,10026.0,3319.0,3.0,...,0,0,0,BATH BEACH,1,2014-10-27,510000,1,0,11214
4,4.0,4.0,270958.0,upper west side,130 MANHATTAN AVENUE,2012-12,2100000.0,10025.0,8155.0,4.0,...,0,0,0,BATH BEACH,1,2015-12-15,520000,1,0,11214
5,5.0,5.0,270958.0,upper west side,60 WEST 106 STREET,2012-08,540000.0,10025.0,2124.0,5.0,...,0,0,0,BATH BEACH,1,2017-06-27,515000,1,0,11214
6,6.0,6.0,270958.0,upper west side,60 WEST 106 STREET,2013-02,575000.0,10025.0,2128.0,6.0,...,0,0,0,BATH BEACH,1,2017-10-18,650000,1,0,11214
7,7.0,7.0,270958.0,upper west side,60 WEST 106 STREET,2014-11,735000.0,10025.0,2469.0,7.0,...,0,2132,2417,BATH BEACH,1,2012-08-24,508000,1,1930,11214
8,8.0,8.0,270958.0,upper west side,60 WEST 107 STREET,2012-09,446160.0,10025.0,1738.0,8.0,...,0,0,0,BATH BEACH,1,2013-08-22,465000,1,2007,11214
9,9.0,9.0,270958.0,upper west side,60 WEST 107 STREET,2012-04,400530.0,10025.0,1545.0,9.0,...,0,0,0,BATH BEACH,1,2014-12-10,480000,1,2007,11214


In [90]:
len(df1) + len(df2) + len(df3)

182563

In [138]:
def cleaning():
    df['RegionID'] = df['RegionID'].fillna(df['RegionID'].mode(), inplace=True)
    df['neighborhood'].fillna(df['neighborhood'].mode(), inplace=True)
    # df['addr'].fillna()
    df['sale_date'].fillna(df['sale_date'].mode(), inplace=True)
    df['sale_price'].fillna(df['sale_price'].mean())
    # df['zip_code'].fillna(df['zip_code'].median())
    df['rent_estimate'].fillna(df['rent_estimate'].mean())
    # df['address'].fillna()
    df['num_within_dist'].fillna(df['num_within_dist'].mode())
    df['station'].fillna(df['station'].mode())
    df['line'].fillna(df['line'].mode())
    df['geometry'].fillna(df['geometry'].mode())
    df['stat_lng'].fillna(df['stat_lng'].median())
    df['stat_lat'].fillna(df['stat_lat'].median())
    df['dist_miles'].fillna(df['dist_miles'].mean())
#     # df['address'].fillna()
#     df['lng'].fillna(df['lng'].mode())
#     df['lat'].fillna(df['lat'].mode())
#     # df['apartment_number'].fillna(df['apartment_number'].mo)
#     df['borough'].fillna(value=df['borough'].mode())
#     df['building_class_category'].fillna(df['building_class_category'].mode())
#     df['commercial_units'].fillna(df['commercial_units'].mode())
#     df['gross_square_feet'].fillna(df['gross_square_feet'].mean())
#     df['land_square_feet'].fillna(df['land_square_feet'].mean())
#     df['neighborhood'].fillna(df['neighborhood'].mode())
#     df['residential_units'].fillna(df['residential_units'].mode()) 
#     df['sale_date'].fillna(df['sale_date'].mode())
#     df['sale_price'].fillna(df['sale_price'].mean())
#     df['total_units'].fillna(df['total_units'].mode())
#     df['year_built'].fillna(df['year_built'].median())
#     # df['zip_code'].fillna(df['zip_code'].mode())
    
cleaning()

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73157 entries, 0 to 73156
Data columns (total 39 columns):
Unnamed: 0                 70146 non-null float64
Unnamed: 0.1               70146 non-null float64
RegionID                   0 non-null object
neighborhood               70146 non-null object
addr                       70146 non-null object
sale_date                  70146 non-null object
sale_price                 70146 non-null float64
zip_code                   70146 non-null float64
rent_estimate              70146 non-null float64
Unnamed: 0                 39260 non-null float64
Unnamed: 0.1               39260 non-null float64
address                    39260 non-null object
num_within_dist            39260 non-null float64
station                    39260 non-null object
line                       39260 non-null object
geometry                   39260 non-null object
stat_lng                   39260 non-null float64
stat_lat                   39260 non-null float64
add

,Unnamed: 0,Unnamed: 0.1,RegionID,neighborhood,addr,sale_date,sale_price,zip_code,rent_estimate,address,...,lat,apartment_number,borough,building_class_category,commercial_units,gross_square_feet,land_square_feet,residential_units,total_units,year_built


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182563 entries, 0 to 73156
Data columns (total 30 columns):
RegionID                   70146 non-null float64
Unnamed: 0                 182563 non-null int64
Unnamed: 0.1               182563 non-null int64
add_lat                    39260 non-null float64
add_lng                    39260 non-null float64
addr                       70146 non-null object
address                    112417 non-null object
apartment_number           73157 non-null object
borough                    73157 non-null float64
building_class_category    73157 non-null object
commercial_units           73157 non-null float64
dist_miles                 39260 non-null float64
geometry                   39260 non-null object
gross_square_feet          73157 non-null float64
land_square_feet           73157 non-null float64
lat                        73157 non-null float64
line                       39260 non-null object
lng                        73157 non-null float

In [32]:
df = pd.concat([df1,df2,df3])
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1','add_lat', 'add_lng'], inplace=True)
df.info()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


AttributeError: 'NoneType' object has no attribute 'info'

In [27]:

df.addr = df['addr'].fillna(df['address'], inplace=True)
df

,RegionID,addr,address,apartment_number,borough,building_class_category,commercial_units,dist_miles,geometry,gross_square_feet,...,rent_estimate,residential_units,sale_date,sale_price,stat_lat,stat_lng,station,total_units,year_built,zip_code
0,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2664.0,NaN,2012-05,695000.0,NaN,NaN,NaN,NaN,NaN,10026.0
1,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2575.0,NaN,2012-03,665000.0,NaN,NaN,NaN,NaN,NaN,10026.0
2,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2796.0,NaN,2012-02,721000.0,NaN,NaN,NaN,NaN,NaN,10026.0
3,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3319.0,NaN,2013-08,955000.0,NaN,NaN,NaN,NaN,NaN,10026.0
4,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8155.0,NaN,2012-12,2100000.0,NaN,NaN,NaN,NaN,NaN,10025.0
5,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2124.0,NaN,2012-08,540000.0,NaN,NaN,NaN,NaN,NaN,10025.0
6,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2128.0,NaN,2013-02,575000.0,NaN,NaN,NaN,NaN,NaN,10025.0
7,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2469.0,NaN,2014-11,735000.0,NaN,NaN,NaN,NaN,NaN,10025.0
8,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1738.0,NaN,2012-09,446160.0,NaN,NaN,NaN,NaN,NaN,10025.0
9,270958.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1545.0,NaN,2012-04,400530.0,NaN,NaN,NaN,NaN,NaN,10025.0


In [103]:
df1_unique_addresses_list = list(pd.unique(df1['addr']))
df3_unique_addresses_list = list(pd.unique(df3['address']))

In [ ]:
                         
    
#  for col in df_full.columns:
#     df_full[col].fillna((df_full[col].mean()), inplace=True)

# sub2['income'].fillna((sub2['income'].mean()), inplace=True)
# df_full.apply(lambda x: x.fillna(x.mean()),axis=0)   
    
#     for col in df_full.columns:
#     df_full[col].fillna((df_full[col].mean()), inplace=True)

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q = """SELECT *
         FROM
        df1
        JOIN 
        df2
        USING (address)
        """
    
results = pysqldf(q)
df_12 = pd.DataFrame(results)
df_12

In [ ]:
df1.set_index('address').join(df2.set_index('address'), rsuffix='r')

In [ ]:
df_x = df1.set_index('sale_date')
df_x = df_x.sort_index(ascending=True)

df3_x = df3.set_index('sale_date')
df3_x = df3_x.sort_index(ascending=True)

In [ ]:
df1_unique_addresses_list = list(pd.unique(df1['addr']))
df3_unique_addresses_list = list(pd.unique(df3['address']))

In [ ]:
df2_unique_addresses.loc[pd.unique(df2_unique_addresses['address'])]

In [ ]:
len(df1_unique_addresses)

In [ ]:
len(df3_unique_addresses)

In [ ]:
df1_unique_addresses = df1.drop_duplicates('addr')
df2_unique_addresses = df2.drop_duplicates('address')
df3_unique_addresses = df3.drop_duplicates('address')

In [ ]:
df_full = pd.concat([df1_unique_addresses,df2_unique_addresses,df3_unique_addresses], sort = True)
df_full.info()

In [ ]:
df_full.drop(columns=['add_lat','add_lng','stat_lat','stat_lng'],inplace=True)


'borough','commercial_units','dist_miles','gross_square_feet','land_square_feet','num_within_dist','rent_estimate','residential_units',

In [ ]:
for col in df_full.columns:
    df_full[col].fillna((df_full[col].mean()), inplace=True)

# sub2['income'].fillna((sub2['income'].mean()), inplace=True)
# df_full.apply(lambda x: x.fillna(x.mean()),axis=0)

In [ ]:
df1_unique_addresses.groupby(['neighborhood']).mean()

In [ ]:
df3_unique_addresses.groupby(['neighborhood']).mean()

In [ ]:
joined = pd.concat([df1_unique_addresses, df2_unique_addresses, df3_unique_addresses])
joined.dropna()

In [ ]:
df1_unique_addresses_list == df3_unique_addresses_list

In [ ]:
unique_addresses = []

for x in df1_unique_addresses_list:
    if (x == df3_unique_addresses_list):
        unique_addresses.append(x)

len(unique_addresses)

In [ ]:
df_y = df1.sort_values(by='sale_date', inplace=True)
df3_y = df3.sort_values(by='sale_date', inplace=True)

In [ ]:
merge1=pd.merge(df_x,df3_x, how='inner', left_index=True, right_index=True)

In [ ]:
dfy = df1[['sale_date', 'addr']]
dfy3 = df3[['sale_date', 'address']]

In [ ]:
#by grouping on the 2 columns, this groups rows where A and B values are the same, 
#we call size which returns the number of unique groups:
df1.sort_values(by='sale_date', inplace=True)
df1.groupby(['sale_date', 'addr']).size().reset_index().rename(columns={0:'count'})